In [ ]:
import numpy as np
import pandas as pd
# read the data
df1 = pd.read_csv('2022.06.12.csv')
df2 = pd.read_csv('2022.06.13.csv')
df3 = pd.read_csv('2022.06.14.csv')
df_dataset = pd.concat([df1, df2, df3])
df_dataset.reset_index(drop=True, inplace=True)
df_dataset.head()

FileNotFoundError: ignored

# Reading the dataset

In [ ]:
# read the data
df1 = pd.read_csv('/kaggle/input/luflow-network-intrusion-detection-data-set/2022/06/2022.06.12/2022.06.12.csv')
df2 = pd.read_csv('/kaggle/input/luflow-network-intrusion-detection-data-set/2022/06/2022.06.13/2022.06.13.csv')
df3 = pd.read_csv('/kaggle/input/luflow-network-intrusion-detection-data-set/2022/06/2022.06.14/2022.06.14.csv')

# Merging into 1 dataframe

In [ ]:
df_dataset = pd.concat([df1, df2, df3])
df_dataset.reset_index(drop=True, inplace=True)
len(df_dataset.columns)

16

# Displaying the data

In [ ]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068376 entries, 0 to 1068375
Data columns (total 16 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   avg_ipt        1068376 non-null  float64
 1   bytes_in       1068376 non-null  int64  
 2   bytes_out      1068376 non-null  int64  
 3   dest_ip        1068376 non-null  int64  
 4   dest_port      964168 non-null   float64
 5   entropy        1068376 non-null  float64
 6   num_pkts_out   1068376 non-null  int64  
 7   num_pkts_in    1068376 non-null  int64  
 8   proto          1068376 non-null  int64  
 9   src_ip         1068376 non-null  int64  
 10  src_port       964168 non-null   float64
 11  time_end       1068376 non-null  int64  
 12  time_start     1068376 non-null  int64  
 13  total_entropy  1068376 non-null  float64
 14  label          1068376 non-null  object 
 15  duration       1068376 non-null  float64
dtypes: float64(6), int64(9), object(1)
memory usage: 130.4

In [ ]:
df_dataset = df_dataset.drop(['num_pkts_out', 'num_pkts_in'], axis=1)

In [ ]:
df_dataset.columns

Index(['avg_ipt', 'bytes_in', 'bytes_out', 'dest_ip', 'dest_port', 'entropy',
       'proto', 'src_ip', 'src_port', 'time_end', 'time_start',
       'total_entropy', 'label', 'duration'],
      dtype='object')

# Taking care of missing data, Removing duplicates

In [ ]:
# replace +ve and -ve infinity with NaN
df_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# drop missing values
df_dataset.dropna(inplace=True)

In [ ]:
print(df_dataset.duplicated().sum())

4447


In [ ]:
df_dataset.drop_duplicates(inplace = True)

In [ ]:
print(df_dataset.duplicated().sum())

0


# Implementing binary classification

In [ ]:
df_dataset["label"].value_counts()

benign       516220
outlier      365385
malicious     78116
Name: label, dtype: int64

In [ ]:
old_value = 'outlier'
new_value = 0
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

In [ ]:
old_value = 'malicious'
new_value = 1
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

In [ ]:
old_value = 'benign'
new_value = 2
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

In [ ]:
df_dataset["label"].value_counts()

2    516220
0    365385
1     78116
Name: label, dtype: int64

# Balancing the data (Undersampling)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import class_weight

# RANDOM_STATE_SEED = 12

from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
import plotly.express as px
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# df = df_dataset
# fig = go.Figure(data=[
#     go.Bar(name='Benign',
#            y=df["label"].value_counts().values[[2]],
#            x=['Benign'],
#            text = df["label"].value_counts()[[2]],
#            orientation='v',
#            textposition='outside',),
#     go.Bar(name='Malicious',
#            y=df["label"].value_counts().values[[1]],
#            x=['Malicious'],
#            text = df["label"].value_counts()[[1]],
#            orientation='v',
#            textposition='outside',),
#     go.Bar(name='outlier',
#            y=df["label"].value_counts().values[[0]],
#            x=['outlier'],
#            text = df["label"].value_counts()[[0]],
#            orientation='v',
#            textposition='outside',)
# ])
# # Change the bar mode
# fig.update_layout(
#                   width=800,
#                   height=600,
#                   title=f'Class Distribution',
#                   yaxis_title='Number of attacks',
#                   xaxis_title='Attack Name',)
# iplot(fig)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# df1 = df[df["label"] == 0][:320000]
# df2 = df[df["label"] == 1][:320000]
# df3=  df[df['label'] == 2][:320000]
# df_equal = pd.concat([df1,df2], axis =0)
# print(df_equal['label'].value_counts())
# # df_equal = pd.concat([df_equal,df3], axis =0)
# # print(len(df2))

In [ ]:
# df_equal["label"].value_counts()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
# fig = go.Figure(data=[
#     go.Bar(name='Benign',
#            y=df_equal["label"].value_counts().values[[2]],
#            x=['Benign'],
#            text = df_equal["label"].value_counts()[[2]],
#            orientation='v',
#            textposition='outside',),
#     go.Bar(name='Malicious',
#            y=df_equal["label"].value_counts().values[[1]],
#            x=['Malicious'],
#            text = df_equal["label"].value_counts()[[1]],
#            orientation='v',
#            textposition='outside',),
#     go.Bar(name='outlier',
#            y=df_equal["label"].value_counts().values[[2]],
#            x=['outlier'],
#            text = df_equal["label"].value_counts()[[2]],
#            orientation='v',
#            textposition='outside',)
# ])
# # Change the bar mode
# fig.update_layout(
#                   width=800,
#                   height=600,
#                   title=f'Class Distribution',
#                   yaxis_title='Number of attacks',
#                   xaxis_title='Attack Name',)
# iplot(fig)

# Creating train/test split

In [ ]:
df=df_dataset
train, test = train_test_split(df, test_size=0.2)

In [ ]:
# Number of rows in train dataset
len(train)

767776

In [ ]:
# rows in test dataset
len(test)

191945

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191945 entries, 256777 to 632695
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   avg_ipt        191945 non-null  float64
 1   bytes_in       191945 non-null  int64  
 2   bytes_out      191945 non-null  int64  
 3   dest_ip        191945 non-null  int64  
 4   dest_port      191945 non-null  float64
 5   entropy        191945 non-null  float64
 6   proto          191945 non-null  int64  
 7   src_ip         191945 non-null  int64  
 8   src_port       191945 non-null  float64
 9   time_end       191945 non-null  int64  
 10  time_start     191945 non-null  int64  
 11  total_entropy  191945 non-null  float64
 12  label          191945 non-null  int64  
 13  duration       191945 non-null  float64
dtypes: float64(6), int64(8)
memory usage: 22.0 MB


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 767776 entries, 947627 to 993580
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   avg_ipt        767776 non-null  float64
 1   bytes_in       767776 non-null  int64  
 2   bytes_out      767776 non-null  int64  
 3   dest_ip        767776 non-null  int64  
 4   dest_port      767776 non-null  float64
 5   entropy        767776 non-null  float64
 6   proto          767776 non-null  int64  
 7   src_ip         767776 non-null  int64  
 8   src_port       767776 non-null  float64
 9   time_end       767776 non-null  int64  
 10  time_start     767776 non-null  int64  
 11  total_entropy  767776 non-null  float64
 12  label          767776 non-null  int64  
 13  duration       767776 non-null  float64
dtypes: float64(6), int64(8)
memory usage: 87.9 MB


# Feature scaling

In [ ]:
min_max_scaler = MinMaxScaler().fit(train[['avg_ipt', 'bytes_in', 'bytes_out', 'dest_port', 'entropy','src_port','total_entropy', 'duration']])

In [ ]:
numerical_columns = ['avg_ipt', 'bytes_in', 'bytes_out', 'dest_port', 'entropy', 'src_port','total_entropy', 'duration']

In [ ]:
train[numerical_columns] = min_max_scaler.transform(train[numerical_columns])

In [ ]:
print(len(train))

767776


In [ ]:
test[numerical_columns] = min_max_scaler.transform(test[numerical_columns])

# Counting number of records in each split

In [ ]:
print("Full dataset:\n")
print("Benign: " + str(df["label"].value_counts()[[2]].sum()))
print("Malicious: " + str(df["label"].value_counts()[[1]].sum()))
print("outlier: " + str(df["label"].value_counts()[[0]].sum()))
print("---------------")

print("Training set:\n")
print("Benign: " + str(train["label"].value_counts()[[2]].sum()))
print("Malicious: " + str(train["label"].value_counts()[[1]].sum()))
print("outlier: " + str(train["label"].value_counts()[[0]].sum()))
print("---------------")

print("Test set:\n")
print("Benign: " + str(test["label"].value_counts()[[2]].sum()))
print("Malicious: " + str(test["label"].value_counts()[[1]].sum()))
print("outlier: " + str(test["label"].value_counts()[[0]].sum()))

Full dataset:

Benign: 516220
Malicious: 78116
outlier: 365385
---------------
Training set:

Benign: 413040
Malicious: 62351
outlier: 292385
---------------
Test set:

Benign: 103180
Malicious: 15765
outlier: 73000


# Creating X, y variables

In [ ]:
y_train = np.array(train.pop('label'))# pop removes "Label" from the dataframe
X_train = train.values

In [ ]:
y_test = np.array(test.pop("label")) # pop removes "Label" from the dataframe
X_test = test.values

In [ ]:
# Checking unique values in the label column
df['label'].unique()

array([0, 2, 1])

# Neural Network

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense,LSTM,Conv1D,MaxPool1D,Dropout,Flatten,Reshape
from tensorflow.python.keras.losses import SparseCategoricalCrossentropy
from tensorflow.python.keras.optimizers import Adam

In [ ]:
model = Sequential() # initializing model
#lstm
model.add(Conv1D(512,1, padding="same",input_shape = (X_train.shape[1], 1), activation='relu'))
model.add(MaxPool1D(pool_size=(2)))
model.add(LSTM(units=16,return_sequences=False,dropout=0.2))
model.add(Dense(units=3,activation='softmax'))

In [ ]:
# defining loss function, optimizer, metrics and then compiling model
model.compile(loss=SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 13, 512)           1024      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 6, 512)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 16)                33856     
                                                                 
 dense (Dense)               (None, 3)                 51        
                                                                 
Total params: 34,931
Trainable params: 34,931
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=model.fit(X_train, y_train, epochs=50, batch_size=512, validation_data=(X_test,y_test))

Epoch 1/50
1500/1500 [==============================] - 22s 9ms/step - loss: 0.9077 - accuracy: 0.5371 - val_loss: 0.9071 - val_accuracy: 0.5375
Epoch 2/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.5247 - accuracy: 0.8123 - val_loss: 0.3887 - val_accuracy: 0.8572
Epoch 3/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3746 - accuracy: 0.8573 - val_loss: 0.3619 - val_accuracy: 0.8582
Epoch 4/50
1500/1500 [==============================] - 13s 8ms/step - loss: 0.3588 - accuracy: 0.8589 - val_loss: 0.3531 - val_accuracy: 0.8616
Epoch 5/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3525 - accuracy: 0.8599 - val_loss: 0.3490 - val_accuracy: 0.8605
Epoch 6/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3493 - accuracy: 0.8604 - val_loss: 0.3467 - val_accuracy: 0.8607
Epoch 7/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3479 - accuracy: 0.8603 - val_loss: 0.3463 - val_ac

# Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'entropy', max_depth=5)

parameters = {'max_depth': randint(2, 10), 'min_samples_split': randint(2, 10), 'min_samples_leaf': [1, 2, 3, 4, 5], 'criterion': ['gini', 'entropy']}

random_search = RandomizedSearchCV(dt, param_distributions=parameters, n_iter=20, cv=5, n_jobs=-1)
random_search.fit(X_train, y_train)
# best_accuracy = random_search.best_score_
# best_parameters = random_search.best_params_

# print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
# print("Best Parameters:", best_parameters)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


RandomizedSearchCV(cv=5,
                   estimator=DecisionTreeClassifier(criterion='entropy',
                                                    max_depth=5),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ec5cc3e6a40>,
                                        'min_samples_leaf': [1, 2, 3, 4, 5],
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ec5c03eba90>})

In [ ]:
from sklearn import metrics
# dt = DecisionTreeClassifier(**random_search.best_params_)
dt.fit(X_train,y_train)

# Predicting Train & Test Results
y_dt = dt.predict(X_train)
y_pred_dt = dt.predict(X_test)

# Accuracy
print("Accuracy on Training set: ",metrics.accuracy_score(y_train, y_dt))
print("Accuracy on Testing set: ",metrics.accuracy_score(y_test, y_pred_dt))

Accuracy on Training set:  0.9602748718376193
Accuracy on Testing set:  0.9599624892547344


In [ ]:
print(len(y_dt))
print(f"Malicious: {len(y_dt[y_dt==1])}")
print(f"Outliers: {len(y_dt[y_dt==0])}")
print(f"Benign: {len(y_dt[y_dt==2])}")

767776
Malicious: 56839
Outliers: 298211
Benign: 412726


# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=40)

parameters = {'n_estimators': randint(50, 100), 'max_depth': randint(2, 8), 'min_samples_split': randint(2, 10),
              'min_samples_leaf': randint(1, 10), 'max_features': ['sqrt', 'log2', None]}

# random_search = RandomizedSearchCV(rf, n_iter=20, cv=5, n_jobs=-1)
rf.fit(X_train, y_train)
# best_accuracy = rf.accuracy
# best_parameters = rf.best_params_

# print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
# print("Best Parameters:", best_parameters)

RandomForestClassifier(n_estimators=40)

In [ ]:
# best_rf = RandomForestClassifier(rf)
# best_rf.fit(X_train, y_train)

# Predicting Train & Test Results
y_rf = rf.predict(X_train)
y_pred_rf = rf.predict(X_test)

# Accuracy
print("Accuracy on Training set: ",metrics.accuracy_score(y_train, y_rf))
print("Accuracy on Testing set: ",metrics.accuracy_score(y_test, y_pred_rf))

Accuracy on Training set:  0.999914037427583
Accuracy on Testing set:  0.9916486493526792


In [ ]:
print(rf.predict(X_train[1].reshape(1,-1)))

[0]


# Creating Classification Reports

## Decision Tree

In [ ]:
from sklearn.metrics import classification_report

# generate classification report
target_names = ['0', '1','2']
print(classification_report(y_test, y_pred_dt, target_names=target_names))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95     73000
           1       0.79      0.71      0.75     15765
           2       1.00      1.00      1.00    103180

    accuracy                           0.96    191945
   macro avg       0.91      0.89      0.90    191945
weighted avg       0.96      0.96      0.96    191945



## Random Forest

In [ ]:
from sklearn.metrics import classification_report

# generate classification report
target_names = ['0', '1','2']
print(classification_report(y_test, y_pred_rf, target_names=target_names))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     73000
           1       0.96      0.94      0.95     15765
           2       1.00      1.00      1.00    103180

    accuracy                           0.99    191945
   macro avg       0.98      0.98      0.98    191945
weighted avg       0.99      0.99      0.99    191945



# Saving the Models

In [ ]:
from joblib import dump,load
filename = 'random_forest.joblib'
# linear_svm, knn, decision tree, random forest
model = rf
dump(model, filename)

['random_forest.joblib']

In [ ]:
ml_model=joblib.load('/kaggle/working/random_forest.joblib')

In [ ]:
for i in range(1,15):
    output=ml_model.predict(X_train[i].reshape(1,-1))
    if(output==0):
        print("Data received is an outlier")
    elif(output==1):
        print("Data received is Malicious")
    elif(output==2):
        print("Data received is Benign")

Data received is an outlier
Data received is Benign
Data received is an outlier
Data received is Benign
Data received is an outlier
Data received is Benign
Data received is an outlier
Data received is Benign
Data received is an outlier
Data received is Benign
Data received is an outlier
Data received is Benign
Data received is an outlier
Data received is Malicious
